In [1]:
# Mount your Google drive folder on Colab
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [1]:
import pandas as pd
import nltk
import numpy as np
import re
from nltk.stem import wordnet # to perform lemmitization
#from sklearn.feature_extraction.text import CountVectorizer # to perform bow
from sklearn.feature_extraction.text import TfidfVectorizer # to perform tfidf
from nltk import pos_tag # for parts of speech
from sklearn.metrics import pairwise_distances # to perform cosine similarity
from nltk import word_tokenize #  to create tokens
#from nltk.corpus import stopwords # for stop words

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [4]:
df = pd.read_csv('/content/gdrive/My Drive/wine_tasting_dataprep.csv')

In [4]:
#df = pd.read_csv('/content/gdrive/My Drive/wine_tasting_dataprep(2).csv')

In [5]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
variety_list = list(df['variety'].unique())
winery_list = list(df['winery'].unique())
location_list = list(df['Location'].unique())

In [7]:
print('variety', len(variety_list))
print('winery', len(winery_list))
print('location', len(location_list))

variety 750
winery 19176
location 1783


In [8]:
def VID(text):
  for i in range(len(variety_list)):
    if text == variety_list[i]:
      return i
def WID(text):
  for i in range(len(winery_list)):
    if text == winery_list[i]:
      return i
def LID(text):
  for i in range(len(location_list)):
    if text == location_list[i]:
      return i

In [9]:
df['variety_ID'] = df['variety'].apply(VID)
df['winery_ID'] = df['winery'].apply(WID)
df['location_ID'] =df['Location'].apply(LID)

In [10]:
df.head()

,description,variety,winery,Location,lemmatized_text,variety_ID,winery_ID,location_ID
0,This tremendous 100% varietal wine hails from ...,Cabernet Sauvignon,Heitz,Napa Valley California US,this tremendous varietal wine hail from oakvil...,0,0,0
1,"Ripe aromas of fig, blackberry and cassis are ...",Tinta de Toro,Bodega Carmen Rodríguez,Toro Northern Spain,ripe aroma of fig blackberry and cassis be sof...,1,1,1
2,Mac Watson honors the memory of a wine once ma...,Sauvignon Blanc,Macauley,Knights Valley Sonoma California US,mac watson honor the memory of a wine once mak...,2,2,2
3,"This spent 20 months in 30% new French oak, an...",Pinot Noir,Ponzi,Willamette Valley Oregon US,this spent month in new french oak and incorpo...,3,3,3
4,"This is the top wine from La Bégude, named aft...",Provence red blend,Domaine de la Bégude,Bandol Provence France,this be the top wine from la b gude name after...,4,4,4


In [16]:
df.to_csv('/content/gdrive/My Drive/wine_tasting_dataprep(2).csv')

In [6]:
# using tf-idf
 
tfidf = TfidfVectorizer(max_features=184,min_df=7, max_df=0.8, stop_words=stopwords.words('english')) # initializing tf-idf
x_tfidf = tfidf.fit_transform(df['lemmatized_text']).toarray() # transforming the data to array

In [7]:
variety_id = np.array(list(df['variety_ID']))
winery_id = np.array(list(df['winery_ID']))
location_id = np.array(list(df['location_ID']))

In [8]:
df_tfidf = pd.DataFrame(x_tfidf, columns=tfidf.get_feature_names())

In [9]:
from sklearn.model_selection import train_test_split
Xv_train, Xv_test, yv_train, yv_test = train_test_split(df_tfidf, variety_id, test_size=0.2, random_state=101)
Xw_train, Xw_test, yw_train, yw_test = train_test_split(df_tfidf, winery_id, test_size=0.2, random_state=101)
Xl_train, Xl_test, yl_train, yl_test = train_test_split(df_tfidf, location_id, test_size=0.2, random_state=101)

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
v_forest = RandomForestClassifier(n_estimators=3, random_state=2)
v_forest.fit(Xv_train, yv_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=3,
                       n_jobs=None, oob_score=False, random_state=2, verbose=0,
                       warm_start=False)

In [12]:
print("Accuracy on training set: {:.3f}".format(v_forest.score(Xv_train, yv_train)))
print("Accuracy on test set: {:.3f}".format(v_forest.score(Xv_test, yv_test)))

Accuracy on training set: 0.966
Accuracy on test set: 0.829


In [14]:
import joblib

filename = 'finalized_WTDL_model.sav'
joblib.dump(v_forest, filename)

['finalized_WTDL_model.sav']

In [17]:
from sklearn.naive_bayes import MultinomialNB

In [25]:
v_nb = MultinomialNB(alpha=0.001)
v_nb.fit(Xv_train, yv_train)

MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

In [26]:
print("Accuracy on training set: {:.3f}".format(v_nb.score(Xv_train, yv_train)))
print("Accuracy on test set: {:.3f}".format(v_nb.score(Xv_test, yv_test)))

Accuracy on training set: 0.375
Accuracy on test set: 0.375
